In [4]:
from vertexai.preview.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Image,
    Part,
    HarmBlockThreshold,
    HarmCategory,
)

In [1]:
import os

# Set the environment variable before running the script. You can set it in the terminal or in the code.
# os.environ["BUCKET_UPLOAD_TEMP"] = "<your-bucket>"

In [6]:
from google.cloud import storage

def upload_file_to_temp_bucket(file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_name)
    
    return blob.public_url.replace("https://storage.googleapis.com/", "gs://")

In [5]:
gemini_model = GenerativeModel("gemini-pro-vision")


In [24]:
def generate_catch_phrases(business_goal, local_image_resource):

  prompt_template = """You are a copy writer for a telecom company. You need to write 5 short descriptions(copies) for a given business goal and image in json format

  [business goal]
  {business_goal}

  [image]
  """

  generation_config = GenerationConfig(
      temperature=0.1,
      max_output_tokens=2048,
  )

  safety_config = {
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }

  image_uri = upload_file_to_temp_bucket(local_image_resource)

  prompt = prompt_template.format(business_goal = business_goal)
  image = Part.from_uri(
      uri=image_uri,
      mime_type="image/jpeg",
  )
  contents = [prompt, image]

  responses = gemini_model.generate_content(contents, generation_config=generation_config, 
      safety_settings=safety_config, stream=False)

  return responses

In [25]:
responses = generate_catch_phrases("10대 20대를 위한 키치한 판매 문구. 최소 10자 이상. 30자 미만", 'resources/black_shoes_on_the_beach.jpeg')

In [28]:
result_list = responses.candidates[0].content.parts[0].text.split('\n')

In [29]:
for phrase in result_list:
  print(phrase)

 1. **젊음을 위한 신발, ****젊음을 위한 신발****.**
2. **세련된 디자인, 편안한 착용감, 완벽한 핏.**
3. **10대 20대를 위한 키치한 판매 문구. 최소 10자 이상. 30자 미만.**
4. **슈즈가 필요하세요? 지금 바로 주문하세요!**
5. **최고의 품질, 최저의 가격. 지금 바로 구매하세요!**


In [32]:

def verify_phrase(phrase):

  generation_config = GenerationConfig(
      temperature=0.1,
      max_output_tokens=2048,
  )

  safety_config = {
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }
  
  verifier_prompt = """You are the final marketing copy reviewer. Please check if the given copy has any potential issues when used as a marketing copy - e.g., racism, ageism, sexism - and express your score on it, between 0 and 100 points.

  [phrase]
  {phrase}
  """
  contents = [verifier_prompt.format(phrase = phrase)]
  responses = gemini_model.generate_content(contents, generation_config=generation_config, safety_settings=safety_config, stream=False)
  return responses

In [35]:
for phrase in result_list:
  responses = verify_phrase(phrase)
  print(f"phrase: {phrase}, score: {responses.candidates[0].content.parts[0].text}")
  

phrase:  1. **젊음을 위한 신발, ****젊음을 위한 신발****.**, score: **Score:** 0/100

**Potential Issues:**

* **Ageism:** The phrase implies that only young people should wear these shoes, which could be offensive to older adults.
phrase: 2. **세련된 디자인, 편안한 착용감, 완벽한 핏.**, score: The given phrase does not contain any potential issues related to racism, ageism, or sexism. It is a neutral description of a product's features.

Score: 100/100
phrase: 3. **10대 20대를 위한 키치한 판매 문구. 최소 10자 이상. 30자 미만.**, score: The phrase "10대 20대를 위한 키치한 판매 문구" is potentially ageist as it excludes people outside of the specified
phrase: 4. **슈즈가 필요하세요? 지금 바로 주문하세요!**, score: The given phrase does not contain any potential issues related to racism, ageism, or sexism. It is a simple and straightforward call to action that encourages the reader to order shoes.

Score: 100/100
phrase: 5. **최고의 품질, 최저의 가격. 지금 바로 구매하세요!**, score: The phrase "최고의 품질, 최저의 가격. 지금 바로 구매하세요!" does not contain any potential issues related to racism, age

In [36]:
def check_creativity(phrase):
  generation_config = GenerationConfig(
      temperature=0.1,
      max_output_tokens=2048,
  )

  safety_config = {
      HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
      HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  }
  
  verifier_prompt = """You are the final marketing copy reviewer. Please express, on a scale of 0 (low) to 100 (high), whether the given sentence is creative and conveys enough meaning to customers when used as a marketing copy.

  [phrase]
  {phrase}
  """
  contents = [verifier_prompt.format(phrase = phrase)]
  responses = gemini_model.generate_content(contents, generation_config=generation_config, safety_settings=safety_config, stream=False)
  return responses

In [37]:
for phrase in result_list:
  responses = check_creativity(phrase)
  print(f"phrase: {phrase}, score: {responses.candidates[0].content.parts[0].text}")  

phrase:  1. **젊음을 위한 신발, ****젊음을 위한 신발****.**, score: 50
phrase: 2. **세련된 디자인, 편안한 착용감, 완벽한 핏.**, score: 80
phrase: 3. **10대 20대를 위한 키치한 판매 문구. 최소 10자 이상. 30자 미만.**, score: 0
phrase: 4. **슈즈가 필요하세요? 지금 바로 주문하세요!**, score: 50
phrase: 5. **최고의 품질, 최저의 가격. 지금 바로 구매하세요!**, score: 50
